In [ ]:
# demonstration of how points can be matched between
# multiple videos of a single striking ant
# this is temporary and/or needs to be cleaned up

In [ ]:
from hsflfm.processing import Aligner
import numpy as np
from matplotlib import pyplot as plt
from hsflfm.processing import match_points_between_images, get_point_locations
from hsflfm.util import procrustes_analysis

import cv2

In [ ]:
specimen_name = "20220422_OB_1"
#specimen_name = "20240418_OB_1_rp"
#specimen_name = "20240506_OB_7"
#specimen_name = "20220422_OB_1"
#specimen_name = "20240507_OB_2"
#specimen_name = "20240506_OB_1"



aligner = Aligner(specimen_name)

In [ ]:
aligner.stored_point_numbers

In [ ]:
aligner.data_manager.strike_numbers

In [ ]:
import numpy as np
import scipy.signal

def cross_image(im1_gray, im2_gray):
   # get rid of the color channels by performing a grayscale transform
   # the type cast into 'float' is to avoid overflows
   #im1_gray = np.sum(im1.astype('float'), axis=2)
   #im2_gray = np.sum(im2.astype('float'), axis=2)

   # get rid of the averages, otherwise the results are not good
   im1_gray -= np.mean(im1_gray)
   im2_gray -= np.mean(im2_gray)

   # calculate the correlation image; note the flipping of onw of the images
   corr_image = scipy.signal.fftconvolve(im1_gray, im2_gray[::-1,::-1], mode='same')
   unraveled = np.unravel_index(np.argmax(corr_image), corr_image.shape)
   middle = corr_image.shape[0] / 2, corr_image.shape[1] / 2
   return unraveled[0] - middle[0], unraveled[1] - middle[1]

def align_strike(
        self,
        strike_number,
        start_strike=None,
        match_sequentially=True,
        flow_parameters=None,
        reshift=True,
    ):
        run = True
        bad_numbers = []

        threshold = self.alignment_settings["point_error_cutoff"]

        if not match_sequentially and start_strike is None:
            start_strike = 1
        elif match_sequentially and start_strike is None:
            start_strike = max(1, strike_number - 1)

        if start_strike not in self.stored_match_points:
            self.align_strike(start_strike, match_sequentially=match_sequentially)

        strike_match_points = {}
        prev_match_points = self.stored_match_points[start_strike]
        prev_images = self.data_manager.get_start_images(strike_number=start_strike)
        new_images = self.data_manager.get_start_images(strike_number)

        for key, prev_image in prev_images.items():
            new_image = new_images[key]
            pmp = np.asarray(prev_match_points[key])[:, :2]
            
            if reshift:
                plt.figure()
                plt.imshow(prev_image - new_image)
                plt.scatter(pmp[:, 1], pmp[:, 0])
                plt.title("original")
                # this could get thrown in by consistent background
                # so we'll crop in around the actual points
                # buffers = {
                #     0: [65, 10, 35, 10],
                #     1: [0, 45, 0, 15],
                #     2: [45, 10,35, 10]
                # }
                # b = buffers[key]

                rough_shift = (0, 0)#(-50, -15)
                translation_matrix = np.float32([[1, 0, rough_shift[0]], [0, 1, rough_shift[1]]])
                prev_image = cv2.warpAffine(
                    prev_image.T, translation_matrix, prev_image.shape[:2]
                ).T

                plt.figure()
                plt.imshow(prev_image - new_image)
                plt.title("rough aligned")

                pmp = pmp + rough_shift
                
                b = [15, 15, 15, 15]
                minx = int(np.min(pmp[:, 0]) - b[0]) 
                maxx = int(np.max(pmp[:, 0]) + b[1]) 
                miny = int(np.min(pmp[:, 1]) - b[2]) 
                maxy = int(np.max(pmp[:, 1]) + b[3]) 
                print(minx, maxx, miny, maxy)
                minx = max(minx, 0)
                maxx = min(maxx, prev_image.shape[0]) 
                miny = max(miny, 0)
                maxy = min(maxy, prev_image.shape[1])
                
                i0 = prev_image[minx:maxx, miny:maxy].copy()
                i1 = new_image[minx:maxx, miny:maxy].copy()

                plt.figure()
                plt.imshow(i0 - i1)
                plt.title("cropped")
                
                shiftx, shifty = cross_image(i1, i0)
                print(shiftx, shifty)

                # SOMETHING IS WEIRD
                # with the order we're using either shiftx, shifty or rough_shift
                #shiftx = 50 
                #shifty = -10
                translation_matrix = np.float32([[1, 0, shiftx], [0, 1, shifty]])
                prev_image = cv2.warpAffine(
                    prev_image.T, translation_matrix, prev_image.shape[:2]
                ).T
                pmp = pmp + [shiftx, shifty]

            new_points = match_points_between_images(
                prev_image,
                new_image,
                pmp,
                flow_parameters=flow_parameters,
            )

            plt.figure()
            plt.title("fine aligned")
            plt.imshow(prev_image - new_image)

            strike_match_points[key] = new_points

        # convert to camera points
        strike_locations = get_point_locations(self.system, strike_match_points)
        prev_strike_locations = get_point_locations(self.system, prev_match_points)

        strike_point_indices = np.arange(len(prev_match_points[2]))

        while run:
            # fit a transformation between the two
            A_cam2_to_cam1, _, transformed_points = procrustes_analysis(
                strike_locations[strike_point_indices],
                prev_strike_locations[strike_point_indices],
                allow_scale=False,
            )

            # look at error
            diff = np.linalg.norm(
                prev_strike_locations[strike_point_indices] - transformed_points,
                axis=1,
            )
            bp = np.where(diff > threshold)[0]

            if len(bp) > 0:
                worst_index = np.argsort(diff)[-1]
                bad_numbers.append(int(strike_point_indices[worst_index]))
                strike_point_indices = np.delete(strike_point_indices, worst_index)
            else:
                run = False

        for key, item in strike_match_points.items():
            strike_match_points[key] = np.asarray(item)[strike_point_indices]

        self.stored_match_points[strike_number] = strike_match_points
        prev_point_numbers = self.stored_point_numbers[start_strike]

        point_numbers = prev_point_numbers[strike_point_indices]
        self.stored_point_numbers[strike_number] = point_numbers

        return A_cam2_to_cam1, strike_match_points, point_numbers, bad_numbers, start_strike

In [ ]:
flow_parameters = {
    "pyr_scale": 0.9,
    "levels": 15,
    "winsize": 5,
    "iterations": 5,
    "poly_n": 3,
    "poly_sigma": 0.8,
    "flags": 0,
}

# flow_parameters = {
#     "pyr_scale": 0.5,
#     "levels": 15,
#     "winsize": 35,
#     "iterations": 15,
#     "poly_n": 7,
#     "poly_sigma": 1.2,
#     "flags": 0,
# }
#flow_parameters = None
s = 7
res = align_strike(
    aligner, s, reshift=True, flow_parameters=flow_parameters, start_strike=None,
    #rough_flow_parameters=rough_flow_parameters
)

In [ ]:
len(aligner.stored_point_numbers[1])

In [ ]:
len(aligner.stored_point_numbers[s])

In [ ]:
cam_num = 2
sn0 = 6
sn1 = 7



i0 = aligner.data_manager.get_start_images(strike_number=sn0)[cam_num]
i1 = aligner.data_manager.get_start_images(strike_number=sn1)[cam_num]

In [ ]:
fig, axes = plt.subplots(1, 2)
axes[0].imshow(i0, cmap="gray")
axes[1].imshow(i1, cmap="gray")

for ax in axes:
    ax.set_xticks([])
    ax.set_yticks([])
plt.tight_layout()

In [ ]:
diff = i0 - i1
p0 = aligner.stored_match_points[sn0][cam_num]
p1 = aligner.stored_match_points[sn1][cam_num]
plt.imshow(diff)
plt.scatter(p0[:, 1], p0[:, 0], s=5, color="white", alpha=0.5)
plt.scatter(p1[:, 1], p1[:, 0], color="black", s=5, alpha=0.5)

# plt.xlim(60, 130)
# plt.ylim(200, 100)

fig, (ax0, ax1) = plt.subplots(1, 2)
ax0.imshow(i0, cmap='gray')
ax0.scatter(p0[:, 1], p0[:, 0], s=5, color='white')
ax0.set_xticks([])
ax0.set_yticks([])
ax0.set_ylim(200, 50)
ax0.set_xlim(30, 100)

ax1.imshow(i1, cmap='gray')
ax1.scatter(p1[:, 1], p1[:, 0], s=5, color='black')
ax1.set_xticks([])
ax1.set_yticks([])
ax1.set_ylim(200, 50)
ax1.set_xlim(30, 100)

In [ ]:
# potential ants of concern
# 20240506_OB_1
# 20240417 OB 1 and 2

In [ ]:
i0c = i0[50:190, 10:120]
i1c = i1[50:190, 10:120]
print(cross_image(i0c, i1c))
print(cross_image(i0, i1))

plt.imshow(i0c)
plt.figure()
plt.imshow(i1c)

In [ ]:
def correlate_images(im1_gray, im2_gray):
    # get rid of the averages, otherwise the results are not good
    im1_gray -= np.mean(im1_gray)
    im2_gray -= np.mean(im2_gray)

    # calculate the correlation image; note the flipping of onw of the images
    corr_image = scipy.signal.fftconvolve(im1_gray, im2_gray[::-1, ::-1], mode="same")
    return corr_image
from skimage.transform import warp_polar
def get_angular_difference(im0, im1):
    i0p = warp_polar(im0) 
    i1p = warp_polar(im1)

    i1p -= np.mean(i1p)
    i0p -= np.mean(i0p)
    corr_image = scipy.signal.fftconvolve(i0p, i1p[::-1, ::-1], mode="same")
    unraveled = np.unravel_index(np.argmax(corr_image), corr_image.shape)
    return 180 - unraveled[0]



i0c = i0[50:190, 10:120]
i1c = i1[50:190, 10:120]
plt.imshow(i0c - i1c)

angle = get_angular_difference(i0c, i1c)
print(angle)

from skimage.transform import warp, AffineTransform
transform = AffineTransform(rotation=np.deg2rad(-angle))
i0c = warp(i0c, transform)

plt.figure()
plt.imshow(i0c - i1c)

shifty, shiftx = cross_image(i1c, i0c)
print(shiftx, shifty)

from skimage.transform import warp, AffineTransform
transform = AffineTransform(translation=(-shiftx, -shifty))#, rotation=np.deg2rad(-angle))

i0ct = warp(i0c, transform)
plt.figure()
plt.imshow(i0ct - i1c)

print(cross_image(i0ct, i1c))
print(get_angular_difference(i0ct, i1c))

In [ ]:
import cv2
import numpy as np

def estimate_affine_transform(img1, img2):
    """
    Estimate an affine transform between two images using OpenCV feature matching.

    Parameters:
        img1 (ndarray): Reference image.
        img2 (ndarray): Image to align.

    Returns:
        ndarray: The affine transformation matrix (2x3) or None if estimation fails.
    """
    # Detect ORB keypoints and descriptors
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)

    # Match descriptors using BFMatcher
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)

    # Extract matched keypoints
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    # Estimate affine transform
    M, inliers = cv2.estimateAffinePartial2D(src_pts, dst_pts)
    return M

# Example Usage:
# img1 = cv2.imread('image1.jpg', cv2.IMREAD_GRAYSCALE)
# img2 = cv2.imread('image2.jpg', cv2.IMREAD_GRAYSCALE)
# M = estimate_affine_transform(img1, img2)

# Apply the transform
# if M is not None:
#     rows, cols = img1.shape
#     aligned_img = cv2.warpAffine(img2, M, (cols, rows))
#     cv2.imshow("Aligned Image", aligned_img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

M = estimate_affine_transform(i0c.astype(np.uint8), i1c.astype(np.uint8))

cols, rows = i1c.shape
plt.imshow(cv2.warpAffine(i1c, M, (rows, cols)) - i0c)



In [ ]:
def correlate_images(im1_gray, im2_gray):
    # get rid of the averages, otherwise the results are not good
    im1_gray -= np.mean(im1_gray)
    im2_gray -= np.mean(im2_gray)

    # calculate the correlation image; note the flipping of onw of the images
    corr_image = scipy.signal.fftconvolve(im1_gray, im2_gray[::-1, ::-1], mode="same")
    return corr_image

In [ ]:
corr_img = correlate_images(i0, i1)
print(corr_img.shape)

In [ ]:
plt.imshow(corr_img)

In [ ]:
np.unravel_index(np.argmax(corr_img), corr_img.shape)

In [ ]:
print(cross_image(i0c, i1c))

In [ ]:
import numpy as np
from scipy.fft import fft2, ifft2, fftshift
from scipy.ndimage import fourier_shift
from skimage.transform import warp, AffineTransform

def approximate_affine_transform(img1, img2):
    """
    Estimate an approximate affine transform between two images using Fourier space.

    Parameters:
        img1 (ndarray): The first input image (reference).
        img2 (ndarray): The second input image to align to the reference.

    Returns:
        AffineTransform: An AffineTransform object with the estimated parameters.
    """
    # Ensure images are the same size
    assert img1.shape == img2.shape, "Images must be of the same shape."

    # Compute Fourier transforms of both images
    F1 = fft2(img1)
    F2 = fft2(img2)
    
    # Compute cross-power spectrum
    R = (F1 * F2.conj()) / np.abs(F1 * F2.conj())
    R[np.isnan(R)] = 0  # Handle division by zero
    corr = np.abs(ifft2(R))

    plt.imshow(corr)
    
    # Find peak correlation for translation
    shift = np.array(np.unravel_index(np.argmax(corr), corr.shape)) - np.array(img1.shape) // 2

    # Adjust for wrap-around
    shift = np.mod(shift + np.array(img1.shape) // 2, img1.shape) - np.array(img1.shape) // 2

    # Perform log-polar transformation to estimate rotation and scale
    log_polar_img1 = warp_to_log_polar(img1)
    log_polar_img2 = warp_to_log_polar(img2)
    
    # Repeat phase correlation in log-polar space
    F1_log = fft2(log_polar_img1)
    F2_log = fft2(log_polar_img2)
    R_log = (F1_log * F2_log.conj()) / np.abs(F1_log * F2_log.conj())
    R_log[np.isnan(R_log)] = 0
    corr_log = np.abs(ifft2(R_log))
    
    log_shift = np.unravel_index(np.argmax(corr_log), corr_log.shape)
    scale_factor = np.exp(log_shift[0] / corr_log.shape[0])
    rotation_angle = -log_shift[1] / corr_log.shape[1] * 360

    # Construct the affine transform
    transform = AffineTransform(scale=(scale_factor, scale_factor), rotation=np.deg2rad(rotation_angle), translation=shift)

    return transform

def warp_to_log_polar(image):
    """
    Helper function to perform log-polar transformation.
    """
    from skimage.transform import warp_polar
    center = (image.shape[0] // 2, image.shape[1] // 2)
    return warp_polar(image, center=center, scaling='log')

# Example Usage:
# img1 = ... # Reference image as a 2D NumPy array
# img2 = ... # Image to align as a 2D NumPy array
# transform = approximate_affine_transform(img1, img2)
# print(transform.params)  # Matrix representation of the affine transform


In [ ]:
from skimage.transform import warp_polar
plt.imshow(warp_polar(i0c))

In [ ]:
print(i0.shape, warp_polar(i0c).shape)

In [ ]:
from hsflfm.util import cross_image
shifty, shiftx = cross_image(i1c, i0c)
print(shiftx, shifty)

translation_matrix = np.float32([[1, 0, shifty], [0, 1, shiftx]])
i0_shifted = cv2.warpAffine(
    i0c.T, translation_matrix, i0c.shape[:2]
).T

i0p = warp_polar(i0_shifted) 
i1p = warp_polar(i1c)
im1_gray = i1p 
im2_gray = i0p
im1_gray -= np.mean(im1_gray)
im2_gray -= np.mean(im2_gray)

# calculate the correlation image; note the flipping of onw of the images
corr_image = scipy.signal.fftconvolve(im1_gray, im2_gray[::-1, ::-1], mode="same")

plt.imshow(corr_image)
unraveled = np.unravel_index(np.argmax(corr_image), corr_image.shape)
print(unraveled)

In [ ]:
plt.imshow(i0_shifted - i1c)

In [ ]:
plt.imshow(i0c - i1c)

In [ ]:
transform = AffineTransform(rotation=np.deg2rad(-2))
from skimage.transform import warp 
i0cr = warp(i0_shifted, transform)
plt.imshow(i0cr)
plt.figure()
plt.imshow(i0cr - i1c)
plt.figure()
plt.imshow(i0_shifted - i1c)

In [ ]:
transform.params

In [ ]:
approximate_affine_transform(i0c, i1c)